In [0]:
--Preprocessing
SELECT * FROM workspace.default.unicorn_companies;
-- Check data type
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'unicorn_companies';
--change data type by create the new df as nunicron_company_clean
CREATE OR REPLACE TABLE unicorn_companies_clean AS
SELECT
  Company,
  TRY_CAST(REPLACE(`Valuation ($B)`, '$', '') AS DECIMAL(10,2)) AS Valuation_B,
  `Date Joined` AS Date_Joined,
  Country,
  City,
  Industry,
  `Select Inverstors` AS Select_Investors,
  TRY_CAST(`Founded Year` AS INT) AS Founded_Year,
  CASE 
    WHEN `Total Raised` LIKE '%B' THEN 
        ROUND(TRY_CAST(REPLACE(REPLACE(`Total Raised`, '$', ''), 'B', '') AS DOUBLE), 5)
    WHEN `Total Raised` LIKE '%M' THEN 
        ROUND(TRY_CAST(REPLACE(REPLACE(`Total Raised`, '$', ''), 'M', '') AS DOUBLE) / 1000, 5)
    ELSE NULL
END AS Total_Raised,
  `Financial Stage` AS Financial_Stage,
  TRY_CAST(`Investors Count` AS INT) AS Investors_Count,
  `Deal Terms` AS Deal_Terms,
  TRY_CAST(`Portfolio Exits` AS INT) AS Portfolio_Exits
FROM unicorn_companies;
---Check data type
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'unicorn_companies_clean';
-- Display table
SELECT * FROM unicorn_companies_clean;
-- 1. Top 10 companies with most Valuation 
SELECT Company, Valuation_B
FROM unicorn_companies_clean
ORDER BY Valuation_B DESC
LIMIT 20;

---2. Which industries have produced the most unicorns over time
SELECT industry, COUNT(Company) AS number_companies, YEAR(DATE_JOINED) AS year
FROM unicorn_companies_clean
GROUP BY industry,YEAR(Date_Joined)
ORDER BY YEAR(Date_Joined) DESC;

SELECT COUNT(company) as number_companies, industry
FROM unicorn_companies_clean
GROUP BY industry
ORDER BY number_companies DESC;
 --3. Which countries are producing the most unicorns top 2, and how is this changing over the years
WITH ranked_countries AS (
  SELECT
    Country,
    EXTRACT(YEAR FROM Date_Joined) AS year,
    COUNT(*) AS number_companies,
    RANK() OVER (
      PARTITION BY EXTRACT(YEAR FROM Date_Joined)
      ORDER BY COUNT(*) DESC
    ) AS rank_position
  FROM unicorn_companies_clean
  GROUP BY Country, EXTRACT(YEAR FROM Date_Joined)
)
SELECT 
  Country,
  year,
  number_companies
FROM ranked_countries
WHERE rank_position <= 2
ORDER BY year DESC, number_companies DESC;
--4. Industries have the highest average unicorn valuation
SELECT Industry, AVG(Valuation_B) as avg_valuation
FROM unicorn_companies_clean
GROUP BY Industry
ORDER BY AVG(Valuation_B) DESC;

--5. Investors appear most frequently in top-valued unicorns?
SELECT 
  investor AS Select_Investors,
  COUNT(*) AS num_unicorns
FROM (
  SELECT 
    explode(split(Select_Investors, ', ')) AS investor
  FROM unicorn_companies_clean
) AS exploded_investors
GROUP BY investor
ORDER BY num_unicorns DESC;
--6. Companies achieved unicorn status with the least total funding
SELECT Company, Total_Raised, Valuation_B, CAST((Valuation_B/Total_Raised) AS decimal(10,4)) AS ratio
FROM unicorn_companies_clean
WHERE Total_Raised IS NOT NULL
ORDER BY ratio ASC;
--7. How many years does it take for a company (from founding to joining the unicorn list)
SELECT Company, (YEAR(Date_Joined) - Founded_Year) AS years_to_unicorn
FROM unicorn_companies_clean
WHERE Founded_Year IS NOT NULL;
--8. Which cities are the biggest unicorn hubs
SELECT City,Country, Count(*) AS number_companies
FROM unicorn_companies_clean
GROUP BY City, Country
ORDER BY number_companies DESC
LIMIT 10;
--9. industries are growing fastest in recent years (after 2018)
WITH yearly_counts AS (
    SELECT 
        Industry,
        EXTRACT(YEAR FROM Date_Joined) AS year_join,
        COUNT(*) AS number_companies
    FROM unicorn_companies_clean
    WHERE EXTRACT(YEAR FROM Date_Joined) >= 2018
    GROUP BY Industry, EXTRACT(YEAR FROM Date_Joined)
)
SELECT 
    Industry,
    year_join,
    number_companies,
    number_companies - LAG(number_companies) OVER (PARTITION BY Industry ORDER BY year_join) AS yoy_growth
FROM yearly_counts
ORDER BY yoy_growth DESC
LIMIT 10 ;
--10. Is there a strong correlation between total funding raised and valuation?
SELECT CORR( Valuation_B, Total_Raised) AS corr_valuation_funding
FROM unicorn_companies_clean;
--11. percentage of unicorns are from each continent or region?
WITH region_map AS (
    SELECT 
        CASE 
            WHEN Country IN ('United States', 'Canada', 'Mexico') THEN 'North America'
            WHEN Country IN ('China', 'India', 'Japan', 'Singapore', 'South Korea', 'Indonesia') THEN 'Asia'
            WHEN Country IN ('United Kingdom', 'Germany', 'France', 'Sweden', 'Netherlands', 'Spain') THEN 'Europe'
            WHEN Country IN ('Brazil', 'Argentina', 'Chile') THEN 'South America'
            WHEN Country IN ('Australia', 'New Zealand') THEN 'Oceania'
            WHEN Country IN ('Nigeria', 'South Africa', 'Egypt', 'Kenya') THEN 'Africa'
            ELSE 'Other'
        END AS Region,
        *
    FROM unicorn_companies_clean
)
SELECT 
    Region,
    COUNT(*) AS number_companies,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM region_map
GROUP BY Region
ORDER BY percentage DESC;
-- 12. average unicorn valuation changed year over year
WITH yearly_avg AS (
    SELECT 
        EXTRACT(YEAR FROM Date_Joined) AS year,
        AVG(Valuation_B) AS avg_valuation_b
    FROM unicorn_companies_clean
    GROUP BY EXTRACT(YEAR FROM Date_Joined)
)
SELECT 
    year,
    ROUND(avg_valuation_b, 2) AS avg_valuation_b,
    ROUND(
        avg_valuation_b - LAG(avg_valuation_b) OVER (ORDER BY year),
        2
    ) AS yoy_change_abs,
    ROUND(
        100.0 * (avg_valuation_b - LAG(avg_valuation_b) OVER (ORDER BY year)) 
        / LAG(avg_valuation_b) OVER (ORDER BY year),
        2
    ) AS yoy_change_pct
FROM yearly_avg
ORDER BY year;
--13. Which investors dominate specific industries
WITH investor_industry AS (
    SELECT
        Industry,
        TRIM(Select_Investors) AS Investor
    FROM unicorn_companies_clean
    LATERAL VIEW explode(
        split(Select_Investors, ',')
    ) AS investor
)
SELECT 
    Industry,
    Investor,
    COUNT(*) AS number_investments,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY Industry), 2) AS pct_within_industry
FROM investor_industry
GROUP BY Industry, Investor
ORDER BY Industry, number_investments DESC;
--14. industries produce high valuations with lower total funding (efficiency ratio)
SELECT 
    Industry,
    ROUND(AVG(Valuation_B / NULLIF(Total_Raised, 0)), 2) AS avg_efficiency_ratio,
    COUNT(*) AS num_companies
FROM unicorn_companies_clean
WHERE Total_Raised IS NOT NULL
  AND Valuation_B IS NOT NULL
GROUP BY Industry
ORDER BY avg_efficiency_ratio DESC;

--15. unicorns are missing funding info (potential data quality or business failure risks)
SELECT 
    COUNT(*) AS missing_records,
    SUM(CASE WHEN Total_Raised IS NULL OR Total_Raised = '' THEN 1 ELSE 0 END) AS missing_total_raised
FROM unicorn_companies_clean
WHERE Total_Raised IS NULL ;

SELECT Industry, Country, COUNT(*) AS missing_count
FROM unicorn_companies_clean
WHERE Total_Raised IS NULL
GROUP BY Industry, Country
ORDER BY missing_count DESC;

--16. Based on industry and country trends, which industries are likely to produce the next wave of unicorns
WITH yearly_industry_country AS (
    SELECT
        Industry,
        Country,
        EXTRACT(YEAR FROM Date_Joined) AS year,
        COUNT(*) AS new_unicorns,
        AVG(Valuation_B) AS avg_valuation_b,
        AVG(Total_Raised) AS avg_funding_b
    FROM unicorn_companies_clean
    WHERE Date_Joined IS NOT NULL
    GROUP BY Industry, Country, EXTRACT(YEAR FROM Date_Joined)
)
SELECT *
FROM yearly_industry_country
ORDER BY Industry, Country, year;


